In [2]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium

import yaml



Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<ipython-input-2-37216a1e6983>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/home/cat/.conda/envs/donato/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
#############################################
######### LOAD SUITE2P AND BINARIZE #########
#############################################

# input directory where Suite2p outputs matlab Fall.mat and all .npy files
#data_dir = '/media/cat/4TB/donato/andres/master_analysis/DON-014266/20230131/plane0'
data_dir = '/media/cat/4TB/donato/andres/master_analysis/'#20230131/plane0'

#
animal_id = "DON-014451"

# initialize calcium object and load suite2p data
c = calcium.Calcium(data_dir, animal_id)
c.verbose = False                          # outputs additional information during processing
c.recompute_binarization = True           # recomputes binarization and other processing steps; False: loads from previous saved locations

# set flags to save matlab and python data
c.save_python = True         # save output as .npz file 
c.save_matlab = False         # save output as .mat file

# manual thresholds for spike detection
c.dff_min = 0.05                  # min %DFF for [ca] burst to considered a spike (default 5%) overwrites percentile threshold parameter
c.percentile_threshold = 0.999999   # this is pretty fixed, we don't change it; we want [ca] bursts that are well outside the "physics-caused"noise
c.maximum_std_of_signal = 0.08     # if std of signal is greater than this, then we have a noisy signal and we don't want to binarize it
                                  #    - this is a very important flag! come see me if you don't understand it

#
c.binarize_data()


# generate standard randomized plots:
c.save_sample_traces()

#
save_image = True
c.show_rasters(save_image)

print (" ... DONE ...")

 Sessions for animal:  DON-014451
(0)   day0
(1)   20230330
(2)   20230331
(3)   20230402
(4)   20230403
(5)   20230404
(6)   20230405-18threshold
(7)   20230406
(8)   20230407
(a)   All sessions
Please select a session to process:


Processing sesssion: day0

BINARIZING:  day0


model filter: remove bleaching or trends: 100%|██████████| 418/418 [00:04<00:00, 92.76it/s]
420it [00:04, 90.35it/s]                         
binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 418/418 [00:03<00:00, 134.97it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 418/418 [00:15<00:00, 26.45it/s]


...saving data...
 ... DONE ...


In [ ]:
###############################################################
######### LOAD BINARZIED DATA AND VISUALIZE CELL ACTIVITY #####
###############################################################
#fname = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/binarized_traces.npz'
# 
#fname = os.path.join(data_dir,'suite2p','plane0','binarized_traces.npz')
fname = os.path.join(data_dir,'binarized_traces.npz')

c = calcium.Calcium()
c.fname = fname
c.data_dir = data_dir
c.recompute_binarization = False
c.load_binarization()

# pick a random cell to visuzlie
cell_id = 204
scale = .3

# 
c.plot_cell_binarization(cell_id, scale)


In [ ]:
# plt.figure()
# plt.plot(c.F[0])
# c.high_cutoff=0.0002
# res = c.low_pass_filter(c.F[:2])
# plt.plot(res[0])
    
# plt.show()

low pass filter: 100%|██████████| 2/2 [00:00<00:00, 1650.00it/s]


In [ ]:
#/media/cat/4TB/donato/DON-002865/DON-002865_20210211/002P-F/tif/suite2p/plane0/binarized_traces.npz
#/media/cat/4TB/donato/DON-002865/DON-002865_20210211/002P-F/tif/suite2p/plane0/suite2p/plane0/binarized_traces.npz

######################################################
############# SHOW RASTERS ###########################
######################################################
fname = os.path.join(data_dir,'suite2p','plane0','binarized_traces.npz')
#data_dir = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/'

c = calcium.Calcium()
c.data_dir = data_dir
c.fname = fname
c.recompute_binarization = False
c.load_binarization()

c.show_rasters()


In [ ]:
import scipy

#
trace = np.load('/home/cat/trace.npy')

#
y = np.histogram(trace, bins = np.arange(-1,1,0.001))

mode = scipy.stats.mode(trace)[0]
mode_empirical = y[1][np.argmax(y[0])]

#
plt.figure()
plt.plot(y[1][:-1],y[0])
plt.plot([mode,mode],[0,4000])
plt.plot([mode_empirical, mode_empirical],[0,4000])

plt.show()

<ipython-input-62-6f56bde31059>:9: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = scipy.stats.mode(trace)[0]
